In [ ]:
sequences = []
import random
i=1
disksize = 2000
while(i<100):
  sequences.append(random.randint(1, disksize)) #generate a sequence of 100 elements
  i = i+1
print(sequences)
inhead=600 #initial position of the head

FCFS (First Come First Served) - works the same way it does for the processor. It completes the processes in the order of their appearance in the queue. The first one gets completed first, the second second and so on.

In [ ]:
totaltimeFCFS = 0
prev = inhead #prev is the last (most recent) position of the head
for i in sequences:
  if i<prev: #this if checks whether the next element is to the right or to the left of the head and calculates its seektime accordingly
    seektime = prev-i
  else:
    seektime = i-prev
  totaltimeFCFS=totaltimeFCFS+seektime
  prev=i
  print(i)
print(f"Total time= {totaltimeFCFS}")

SSTF (Shortest Seek Time First) - its performance is generally better than FCFS. The queque in this one is sorted on the basis of the seek time, which is the amount of time the disk takes to move its head to the given segment. For a sequence of read requests of {1, 7, 9} when the head is for example at 5 they would be accesed in the order {7, 9, 1}.

In [ ]:
import copy
totaltimeSSTF = 0
prev = inhead
all = copy.deepcopy(sequences)
all.sort()
print(all)
posleft = -1
leftel=0
rightel=len(all)
for i in all:
  if i<inhead:
    posleft+=1
    leftel=i
posright=posleft+1
rightel=all[posleft+1]
while True:
  if(leftel!=all[0]) or (rightel!=all[len(all)-1]):
    print(prev)
    if(leftel!=all[0]):
      lefdis=prev-leftel
    else:
      lefdis=-99999999999999 #the very high numbers in else make sure that the later if conditions don't go off when the only elements left are the first and last one
    if(rightel!=all[len(all)-1]):
      rightdis=rightel-prev
    else:
      rightdis=9999999999999
    if lefdis>=rightdis:
      if rightel!=all[len(all)-1]: #the ifs change the elements on the left and right depending on the distances
        seektime = rightdis
        posright+=1
        prev=rightel
        rightel=all[posright]
    if lefdis<=rightdis:
      if (leftel!=all[0]):
        seektime = lefdis
        posleft-=1
        prev=leftel
        leftel=all[posleft]
    totaltimeSSTF+=seektime
  else:
    break
print(prev)
print(f"Total time= {totaltimeSSTF}")

SCAN (Elevator) - this algorithm starts at the head and depending on the chosen direction it moves from the head to the left or right up until reaching the end of the disk, while seeking out the part of the sequence on that side, and then changes direction. With head at 8, direction to the left and the sequence {5, 6, 9} they will be read in the order {6, 5, 9}. Its execution time would be equal to: (8-6)+(6-5)+(5-0)+(9-0) = 17.

In [ ]:
items=copy.deepcopy(sequences)
items.sort()
totaltimeSCAN=0
i=inhead
stamp=inhead
last = items[-1]
#for direction to the left:
while(i>=0):
  if(i in items) or i==0: #the while moves the diskhead by one tick and if it comes across a sequence it removes it from the array
    print(i)
    if(i in items):
      items.remove(i)
    totaltimeSCAN+=(stamp-i)
    stamp=i
  i-=1
stamp=0
while(i<=last):
  if(i in items):  #this is the same, but after the diskhead reaches the end of the disk
    print(i)
    items.remove(i)
    totaltimeSCAN+=(i-stamp)
    stamp=i
  i+=1
print(f"Total time= {totaltimeSCAN}")

C-SCAN (Circular Scan) - works the same way as regular SCAN, but upon reaching the final element of the sequence at one side, it automatically moves to the end of the disk opposite to its direction. So for the sequence {5, 6, 9}, direction to the left and head at 8, it would read the sequence in order {6, 5, 9} as well, but the execution time would depend on the disk size. For it equal to 10 it would be for example: (8-6)+(6-5)+(5-0)+(10-0)+(10-9)=19. As my implementation doesn't rely on arrival time, its total time will always be similair to the total disk size * 2.


In [ ]:
items=copy.deepcopy(sequences)
items.sort()
totaltimeCSCAN=0
i=inhead
stamp=inhead
pos=-1
for j in all:
  if j<inhead:
    pos+=1
last = items[pos+1]
while(i>=0):
  if(i in items) or i==0:
    print(i)
    if(i in items):
      items.remove(i)
    totaltimeCSCAN+=(stamp-i)
    stamp=i
  i-=1
stamp=disksize
i=disksize
totaltimeCSCAN+=disksize #head of the disk moves to its end
while(i>=last):
  if(i in items):
    print(i)
    items.remove(i)
    totaltimeCSCAN+=(stamp-i)
    stamp=i
  i-=1
print(f"Total time= {totaltimeCSCAN}")

EDF (Earliest Deadline First) - this one works pretty much like FCFS, but instead of the position it considers the deadline of the process. Due to this, it's usually takes the longest out of all the processes, as it cares more about the deadline and not efficiency.

In [ ]:
ev=[]
prev=9999999999999999999
totaltimeEDF=0
counter=0
for i in sequences:
  ev.append([i, random.randint(1, 4*disksize)]) #(sequence, dedaline)
  if(ev[counter][1]<prev):
    prev=ev[counter][1] #this line setermines the first element
  counter+=1
ev.sort(key=lambda x: x[1]) #the sequences are sorted based on their deadline...
print(ev)
for i in range (0, len(ev)-1): #...and are conducted according to it
  if ev[i][0]<prev: #this if checks whether the next element is to the right or to the left of the head and calculates its seektime accordingly
    seektime = prev-ev[i][0]
  else:
    seektime = ev[i][0]-prev
  totaltimeEDF=totaltimeEDF+seektime
  prev=i
  print(ev[i][0])
print(f"Total time= {totaltimeEDF}")

FD-SCAN (Feasible Deadline Scan) - this one on the other hand is like SCAN, but using deadlines of the processes instead of the direction. While travelling to the point with the lowest deadline it also starts up all of the processes it encounters on its way. Additionally if the deadline of the process cannot be met, it is not taken into consideration.

In [ ]:
seq=copy.deepcopy(sequences)
bydeadline=copy.deepcopy(ev)
bydeadlineco=copy.deepcopy(ev)
bysequence=copy.deepcopy(sequences)
print(bydeadline)
print(bydeadlineco[0][0])
totaltimeFDSCAN=0
j=0
stamp=inhead
while(len(seq)!=0):
  while(j<len(bydeadlineco)-1):
    i=bydeadlineco[j][0]
    if(stamp-i>0 and bydeadlineco[j][1]-(stamp-i-totaltimeFDSCAN)>0): #the ifs take the process which can be finished before its deadline
      dir="left"                                                                                  #and check its placement relative to the head
      break
    if(i-stamp>0 and bydeadlineco[j][1]-(i-stamp-totaltimeFDSCAN)>0):
      dir="right"
      break
    j+=1
  print(stamp,dir, i)
  if(stamp==i): #this condition goes off when all the remaining process are past the deadline
    break
  t=stamp
  if(dir=="left"):
    while(t>=i): #this is similair to scan, but it's done for both directions
      if(t in seq) or t==0:
        totaltimeFDSCAN+=(stamp-t)
        stamp=t
        print(t)
        seq.remove(t)
      t-=1
  else:
    while(t<=i):
      if(t in seq) or t==0:
        totaltimeFDSCAN+=(t-stamp)
        stamp=t
        print(t)
        seq.remove(t)
      t+=1
  bydeadlineco=[]
  for d in seq: #the for appends the array with processes that are not finished yet
    f=-1
    while f<len(seq)-1:
      f+=1
      if (bydeadline[f][0]==d):
        bydeadlineco.append([bydeadline[f][0], bydeadline[f][1]])
        break
  if(bydeadlineco==[]): #if the array is empty it means all the prcesses were done
    break
  bydeadlineco.sort(key=lambda x: x[1])
print(f"Total time= {totaltimeFDSCAN}")

In [ ]:
print(f"Total time of FCFS = {totaltimeFCFS}")
print(f"Total time of SSTF = {totaltimeSSTF}")
print(f"Total time of SCAN = {totaltimeSCAN}")
print(f"Total time C-SCAN = {totaltimeCSCAN}")
print(f"Total time of EDF= {totaltimeEDF}")
print(f"Total time of FD-SCAN = {totaltimeFDSCAN}")

Total time of FCFS = 65439
Total time of SSTF = 3235
Total time of SCAN = 2588
Total time C-SCAN = 3998
Total time of EDF= 89455
Total time of FD-SCAN = 2579


Most of the time the simulations show that FD-SCAN is the fastest one, and by a lot. Its results would definitely be worse if it went through all the processes even after the deadlines expired. As for the longest one that's usually EDF, which goes through the processes by their deadlines, which similarly to FCFS isn't very efficient. SCAN, C-SCAN and SSTF are usually similar, but the case would be different if I added arrival time.